# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,2.79,87,100,9.20,CL,1696645362
1,1,bilibino,68.0546,166.4372,-8.36,90,96,0.76,RU,1696645459
2,2,tindouf,27.6711,-8.1474,27.25,19,90,6.17,DZ,1696645460
3,3,puerto ayora,-0.7393,-90.3518,23.64,94,81,2.24,EC,1696645460
4,4,waitangi,-43.9535,-176.5597,13.23,81,33,2.68,NZ,1696645460


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_df=city_data_df[(city_data_df['Max Temp']>=22) & (city_data_df['Max Temp']<27)]

# Drop any rows with null values
ideal_temp_df=ideal_temp_df.dropna()

# Display sample data
ideal_temp_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,puerto ayora,-0.7393,-90.3518,23.64,94,81,2.24,EC,1696645460
6,6,tura,25.5198,90.2201,22.58,99,99,1.08,IN,1696645460
15,15,myeik,12.4333,98.6000,24.68,95,100,1.05,MM,1696645462
24,24,margaret river,-33.9500,115.0667,24.10,38,6,3.44,AU,1696645463
27,27,flying fish cove,-10.4217,105.6791,26.82,78,75,5.14,CX,1696645463
...,...,...,...,...,...,...,...,...,...,...
557,557,mora,11.0461,14.1401,24.50,71,93,1.37,CM,1696645548
562,562,padangsidempuan,1.3667,99.2667,26.88,82,99,0.81,ID,1696645548
563,563,freetown,8.4840,-13.2299,24.96,90,100,0.58,SL,1696645549
565,565,stephenville,32.2207,-98.2023,23.43,52,0,1.54,US,1696645549


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_temp_df[["City", "Country", "Lat", "Lng", "Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

/var/folders/3x/qx01f_zs0f56dh_9xblp7nh40000gp/T/ipykernel_7162/816880453.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
3,puerto ayora,EC,-0.7393,-90.3518,94,
6,tura,IN,25.5198,90.2201,99,
15,myeik,MM,12.4333,98.6000,95,
24,margaret river,AU,-33.9500,115.0667,38,
27,flying fish cove,CX,-10.4217,105.6791,78,
...,...,...,...,...,...,...
557,mora,CM,11.0461,14.1401,71,
562,padangsidempuan,ID,1.3667,99.2667,82,
563,freetown,SL,8.4840,-13.2299,90,
565,stephenville,US,32.2207,-98.2023,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
limit = 10

params = {
    "categories" : "accommodation.hotel",
    "limit" : limit,
    "apiKey":geoapify_key    
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df.loc[index,'Lat']
    longitude=hotel_df.loc[index,'Lng']
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
tura - nearest hotel: No hotel found
myeik - nearest hotel: Green Eyes Hotel
margaret river - nearest hotel: Margaret River Hotel
flying fish cove - nearest hotel: Christmas Island Lodge
pentecoste - nearest hotel: No hotel found
tepalcatepec - nearest hotel: No hotel found
caxito - nearest hotel: IU Hotel
kone - nearest hotel: Pacifik Appartels
mandal - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
teknaf - nearest hotel: No hotel found
atakpame - nearest hotel: Hôtel St Louis
alofi - nearest hotel: Matavai Resort
avarua - nearest hotel: Paradise Inn
nuku'alofa - nearest hotel: City Hotel
aracoiaba - nearest hotel: Hotel Pasre Cícero
west island - nearest hotel: Cocos Village Bungalows
antsiranana - nearest hotel: Central
holualoa - nearest hotel: Kona Hotel
guercif - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
tangalle - nearest hotel: Coco Tangal

,City,Country,Lat,Lng,Humidity,Hotel Name
3,puerto ayora,EC,-0.7393,-90.3518,94,Hostal La Mirada De Solitario George
6,tura,IN,25.5198,90.2201,99,No hotel found
15,myeik,MM,12.4333,98.6000,95,Green Eyes Hotel
24,margaret river,AU,-33.9500,115.0667,38,Margaret River Hotel
27,flying fish cove,CX,-10.4217,105.6791,78,Christmas Island Lodge
...,...,...,...,...,...,...
557,mora,CM,11.0461,14.1401,71,Hôtel Podoko
562,padangsidempuan,ID,1.3667,99.2667,82,Hotel Mega Permata
563,freetown,SL,8.4840,-13.2299,90,Formal Leona hotel
565,stephenville,US,32.2207,-98.2023,52,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=["Country","Hotel Name"]
)
# Display the map
map_plot



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)